In [13]:
import pickle
import torch
from torch_geometric.data import Data
import networkx as nx
from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import negative_sampling


In [14]:
follower = {}
friend = {}
e_tweet = {}
m_tweet = {}
r_tweet = {}

with open('./Unfollower/15weeks_follower_dict.pkl', 'rb') as f:
    follower = pickle.load(f)

with open('./Unfollower/15weeks_friend_dict.pkl', 'rb') as f:
    friend = pickle.load(f)
    
with open('./Unfollower/e_tweet_dict.pkl', 'rb') as f:
    e_tweet = pickle.load(f)
    
with open('./Unfollower/m_tweet_dict.pkl', 'rb') as f:
    m_tweet = pickle.load(f)
    
with open('./Unfollower/r_tweet_dict.pkl', 'rb') as f:
    r_tweet = pickle.load(f)

In [15]:
train_range = (0,9)
test_range = (10,14)


# make a networkx graph in order to find the clustering coefficient of the nodes
G = nx.Graph()
for key in follower:
    if len(follower[key][train_range[1]]) == 2:
        for f in follower[key][train_range[1]][1]:
            G.add_edge(key,f)
            
for key in friend:
    if len(friend[key][train_range[1]]) == 2:
        for f in friend[key][train_range[1]][1]:
            G.add_edge(f,key)
            
# make another for the test set
G2 = nx.Graph()
for key in follower:
    if len(follower[key][test_range[1]]) == 2:
        for f in follower[key][test_range[1]][1]:
            G2.add_edge(key,f)
            
for key in friend:
    if len(friend[key][test_range[1]]) == 2:
        for f in friend[key][test_range[1]][1]:
            G2.add_edge(f,key)

# make a mapping of the person's id to their index in the node list
# The index will be the index of their x values when making the Data object
id_to_in = {}
in_to_id = list(follower.keys())
i = 0
for key in follower:
    id_to_in[key] = i
    i += 1

In [16]:
nodes1 = []
nodes2 = []

# mapping of edge tuple to index in edge_index tensor
edge_to_in = {}

# Create edge lists for the train Data object
for key in follower:
    if len(follower[key][train_range[1]]) == 2:
        for f in follower[key][train_range[1]][1]:
            edge_to_in[(key,f)] = len(nodes1)
            nodes1.append(id_to_in[key])
            nodes2.append(id_to_in[f])
            
for key in friend:
    if len(friend[key][train_range[1]]) == 2:
        for f in friend[key][train_range[1]][1]:
            edge_to_in[(f,key)] = len(nodes1)
            nodes1.append(id_to_in[f])
            nodes2.append(id_to_in[key])
            
nodes3 = []
nodes4 = []
edge_to_in2 = {}

    
# Create edge lists for the test Data object
for key in follower:
    if len(follower[key][test_range[1]]) == 2:
        for f in follower[key][test_range[1]][1]:
            edge_to_in2[(key,f)] = len(nodes3)
            nodes3.append(id_to_in[key])
            nodes4.append(id_to_in[f])
            
for key in friend:
    if len(friend[key][test_range[1]]) == 2:
        for f in friend[key][test_range[1]][1]:
            edge_to_in2[(key,f)] = len(nodes3)
            nodes3.append(id_to_in[f])
            nodes4.append(id_to_in[key])

In [17]:
# calculate the clustering coefficients
cluster_coeffs = nx.clustering(G)

cluster_coeffs2 = nx.clustering(G2)

In [20]:

# put clustering coefficients in feature array
n = len(list(id_to_in.keys()))
x = [[0.0,0.0]] * n
x2 = [[0.0,0.0]] * n

for i in range(n):
    key = str(in_to_id[i])
    if key in cluster_coeffs:
        x[i][0] = cluster_coeffs[key]
        
for i in range(n):
    key = str(in_to_id[i])
    if key in cluster_coeffs2:
        x2[i][0] = cluster_coeffs2[key]


In [21]:
# add number of tweets to feature list x

for i in range(n):
    key = str(in_to_id[i])
    for i in range(train_range[1]):
        if key in e_tweet['train'][i+1]:
            x[i][1] += len(e_tweet['train'][i+1][key])
            
for i in range(n):
    key = str(in_to_id[i])
    for i in range(test_range[0],test_range[1]):
        if key in e_tweet['test'][i+1]:
            x2[i][1] += len(e_tweet['test'][i+1][key])


In [22]:
        
x = torch.tensor(x, dtype=torch.float)
x2 = torch.tensor(x2, dtype=torch.float)
edge_index = torch.tensor([nodes1,nodes2], dtype=torch.long)
edge_index2 = torch.tensor([nodes3,nodes4], dtype=torch.long)
n_edges = len(nodes1)
n_edges2 = len(nodes3)
edge_label = torch.tensor([1.0]*n_edges, dtype=torch.float)
edge_label2 = torch.tensor([1.0]*n_edges2, dtype=torch.float)
# ?? ROX_AUC giving error because there is only 1 class (everything is 1), I added a 0 here to
# get around this ... 
edge_label2[0] = 0 
train_data = Data(x=x, edge_index=edge_index, edge_label_index=edge_index, edge_label=edge_label)
test_data = Data(x=x2, edge_index=edge_index2, edge_label_index=edge_index2, edge_label=edge_label2)

# delete dicts to free memory
del follower
del friend
del e_tweet
del m_tweet
del r_tweet
del x
del edge_index
del edge_label
del x2
del edge_index2
del edge_label2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data = train_data.to(device)


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        heads = 8
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=.6)
        self.conv2 = GATConv(hidden_channels*heads, out_channels, heads=1, dropout=.6, concat=True)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

num_features = 2
model = Net(num_features, 16, num_features).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=.01, weight_decay=.1)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    #val_auc = test(val_data)
    test_auc = test(test_data)
    if test_auc > best_test_auc:
        best_test_auc = test_auc
    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
    #       f'Test: {test_auc:.4f}')
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test: {test_auc:.4f}')

print(f'Best Test: {best_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 50843901952.0000, Test: 0.5000
Epoch: 002, Loss: 14696592384.0000, Test: 0.5000
Epoch: 003, Loss: 14234330112.0000, Test: 0.5000
